# Data preparation

In [ ]:
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.pipeline  import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.mixture import GaussianMixture
import numpy as np
from sklearn.neighbors import NearestNeighbors
# import smogn
from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN, BorderlineSMOTE, SVMSMOTE

## Set path to folder with data

In [ ]:
folder_path = Path('../diabetes_project/cleaned_data/HUPA-UCM-cleaned_data')

## Load patients info and check duplicates

In [ ]:
# Check folder and file existence
if not os.path.exists(folder_path):
    raise FileNotFoundError(f"Directory {folder_path} does not exist.")

## Load time series data and combine and preprocess data

In [ ]:
all_files = [folder_path / f for f in os.listdir(folder_path) if f.endswith('.csv')]
data_list = []
for file in all_files:
    temp_data = pd.read_csv(file, delimiter=';')
    temp_data['person_id'] = file.stem
    data_list.append(temp_data)

df = pd.concat(data_list, ignore_index=True).drop_duplicates()

## Convert time and handle errors

In [ ]:
df['time'] = pd.to_datetime(df['time'], errors='coerce')
df = df.dropna(subset=['time'])
df['minute'] = df['time'].dt.minute
df['hour_of_day'] = df['time'].dt.hour
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day.clip(1, df['time'].dt.days_in_month)
df.columns

## Anomaly checks

In [ ]:
# Heart rate check
df = df[(df['heart_rate'] > 40) & (df['heart_rate'] < 200)]
# Glucose range check
df = df[(df['glucose'] >= 0) & (df['glucose'] <= 500)]
# Minute range check
df = df[(df['minute'] >= 0) & (df['minute'] <= 59)]
# Hour of day range check
df = df[(df['hour_of_day'] >= 0) & (df['hour_of_day'] <= 23)]
# Month range check
df = df[(df['month'] >= 1) & (df['month'] <= 12)]
# Age range check

## Check target variable

In [ ]:
if df['basal_rate'].isna().any():
    raise ValueError("Target variable 'basal_rate' contains missing values!")

## Analyse of data

In [ ]:
df.head(8)

In [ ]:
plt.figure(figsize=(10, 5))
sns.heatmap(df.drop(columns=['bolus_volume_delivered', 'person_id']).corr(), annot=True, fmt='.2f')

In [ ]:
df.describe()

In [ ]:
# Histogram/Chart for all columns (total amount of HUPA-UCM-full_data)
counts = df.count()
counts

In [ ]:
df.hist(bins=25, figsize=(30, 30), color='green')
plt.show()

## Define features

In [ ]:
features = [
    'glucose', 'calories', 'heart_rate', 'steps',
    'bolus_volume_delivered', 'carb_input', 'minute',
    'hour_of_day', 'month', 'day',
]
print(features)

In [ ]:
X = df[features]
y = df['basal_rate']

# SMOGN

In [ ]:
# df_smogn = smogn.smoter(
#     HUPA-UCM-full_data=df.sample(n=50000, random_state=42).reset_index(drop=True),
#     # HUPA-UCM-full_data=df, # Original HUPA-UCM-full_data frame with HUPA-UCM-full_data
#     y='basal_rate', # Target variable (what to balance)
#     k=5, # Number of neighbors to generate new points
#     pert=0.02, # Percentage of noise added to new HUPA-UCM-full_data
#     samp_method="balance", # Balancing method ("balance" or "extreme")
#     under_samp=True # Whether to further reduce the number of frequent values
# )
#
# X_smogn = df_smogn[features]
# y_smogn = df_smogn['basal_rate']

In [ ]:
# reset_index(drop=True) is like renumbering the pages in a new book. Without it, SMOGN can get "lost" and crash with an error. This is standard practice when working with subselects in Pandas.
# we set df.sample(n=50000, random_state=42).reset_index(drop=True) instead of df

# SMOTER

In [ ]:
def smoter(X, y, tE, o=100, k=5):
    y_median = np.median(y)
    rare_low = np.where((y < y_median) & (y > tE))[0]
    rare_high = np.where((y > y_median) & (y > tE))[0]

    rare_indices = np.concatenate((rare_low, rare_high))
    if len(rare_indices) == 0:
        return X, y

    X_rare = X.iloc[rare_indices]
    y_rare = y.iloc[rare_indices]

    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(X_rare)

    new_X = []
    new_y = []

    for i in range(len(X_rare)):
        neighbors = knn.kneighbors([X_rare.iloc[i]], return_distance=False)[0]
        for _ in range(o // 100):
            neighbor_idx = np.random.choice(neighbors[1:])
            alpha = np.random.rand()
            synthetic_x = X_rare.iloc[i] + alpha * (X_rare.iloc[neighbor_idx] - X_rare.iloc[i])
            d1 = np.linalg.norm(X_rare.iloc[i] - synthetic_x)
            d2 = np.linalg.norm(X_rare.iloc[neighbor_idx] - synthetic_x)
            synthetic_y = (d2 * y_rare.iloc[i] + d1 * y_rare.iloc[neighbor_idx]) / (d1 + d2)

            new_X.append(synthetic_x)
            new_y.append(synthetic_y)

    X_synthetic = pd.DataFrame(new_X, columns=X.columns)
    y_synthetic = pd.Series(new_y)

    X_resampled = pd.concat([X, X_synthetic], ignore_index=True)
    y_resampled = pd.concat([y, y_synthetic], ignore_index=True)

    return X_resampled, y_resampled

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # This method is used to train the normalizer on training HUPA-UCM-full_data.
X_test_scaled = scaler.transform(X_test)
X_final, y_final = smoter(pd.DataFrame(X_train_scaled, columns=features), y_train, tE=np.percentile(y_train, 10))

# Over-sampling samplers (DON'T WORK FOR REGRESSION)

In [ ]:
# SMOTE is applied after splitting the HUPA-UCM-full_data because it should only operate on the training set to avoid HUPA-UCM-full_data leakage.
# In SMOGN, oversampling is first applied on the entire dataset and then splitting. This is because SMOGN uses a strategy that takes into account the distribution of the target variable, and applying it before splitting helps create a more balanced dataset.
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# smote = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=42)
# smote = RandomOverSampler(sampling_strategy='auto', random_state=42)
# smote = KMeansSMOTE(sampling_strategy='auto', k_neighbors=5, random_state=42)
# smote = ADASYN(sampling_strategy='auto', random_state=42)
# smote = SVMSMOTE(sampling_strategy='auto', random_state=42)
# smote = BorderlineSMOTE(sampling_strategy='auto', random_state=42)
smote = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_smote)  # This method is used to train the normalizer on training HUPA-UCM-full_data.
X_test_scaled = scaler.transform(X_test)  # This is important because the test HUPA-UCM-full_data should be scaled in the same way as the training HUPA-UCM-full_data to avoid HUPA-UCM-full_data leakage.

# fit should be used when you want to calculate parameters (like mean and standard deviation) on the HUPA-UCM-full_data to use for scaling.
# transform is used to scale the HUPA-UCM-full_data using the parameters that have already been calculated.
# fit_transform is just a convenience form that first calculates the parameters and then scales the HUPA-UCM-full_data on the training HUPA-UCM-full_data.
# transform on X_test is used to transform the test HUPA-UCM-full_data by the same parameters that were calculated on the training HUPA-UCM-full_data.

## Now calculating summary statistics for each of the columns

In [ ]:
summary = df[features].describe().transpose()

# Predict dose

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Predict dose (SMOGN)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_smogn, y_smogn, test_size=0.2, random_state=42)

## Data scaling

In [ ]:
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# SMOGN (DT)

In [ ]:
model = DecisionTreeRegressor(max_depth=22, min_samples_leaf=2, min_samples_split=2, random_state=42)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

mse_rounded = round(mse, 6)
r2_rounded = round(r2, 6)
mae_rounded = round(mae, 6)

# Output results
print(f'Decision Tree. Mean Squared Error: {mse_rounded}')
print(f'Decision Tree. R²: {r2_rounded}')
print(f'Decision Tree. Mean Absolute Error: {mae_rounded}')

# SMOGN (RF)

In [ ]:
model = RandomForestRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=1, random_state=42)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

mse_rounded = round(mse, 6)
r2_rounded = round(r2, 6)
mae_rounded = round(mae, 6)

# Output results
print(f'Random Forest. Mean Squared Error: {mse_rounded}')
print(f'Random Forest. R²: {r2_rounded}')
print(f'Random Forest. Mean Absolute Error: {mae_rounded}')

# SMOGN (XGB)

In [ ]:
model = XGBRegressor(n_estimators=800, subsample=1.0, reg_lambda=1, reg_alpha=0.1, max_depth=14, learning_rate=0.05,
                     gamma=0, colsample_bytree=0.6)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

mse_rounded = round(mse, 6)
r2_rounded = round(r2, 6)
mae_rounded = round(mae, 6)

# Output results
print(f'XGB. Mean Squared Error: {mse_rounded}')
print(f'XGB. R²: {r2_rounded}')
print(f'XGB. Mean Absolute Error: {mae_rounded}')

# SMOGN (LR)

In [ ]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

mse_rounded = round(mse, 6)
r2_rounded = round(r2, 6)
mae_rounded = round(mae, 6)

# Output results
print(f'Random Forest. Mean Squared Error: {mse_rounded}')
print(f'Random Forest. R²: {r2_rounded}')
print(f'Random Forest. Mean Absolute Error: {mae_rounded}')

# SMOGN (KNN)

In [ ]:
model = KNeighborsRegressor(weights='distance', n_neighbors=7, metric='manhattan')
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

mse_rounded = round(mse, 6)
r2_rounded = round(r2, 6)
mae_rounded = round(mae, 6)

# Output results
print(f'KNN. Mean Squared Error: {mse_rounded}')
print(f'KNN. R²: {r2_rounded}')
print(f'KNN. Mean Absolute Error: {mae_rounded}')

# GMM

In [ ]:
# Augmentation (10% of original size for large dataset)
gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(X_train_scaled)
percentage_quan_sample = 0.1
X_augmented, _ = gmm.sample(n_samples=int(percentage_quan_sample * len(X_train_scaled)))  # 10% of the original volume

# Data Merging
X_final = np.vstack([X_train_scaled, X_augmented])
y_final = np.concatenate([y_train, y_train[:len(X_augmented)]])

### RF

In [ ]:
model = RandomForestRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=1, random_state=42)
model.fit(X_final, y_final)

y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

mse_rounded = round(mse, 6)
r2_rounded = round(r2, 6)
mae_rounded = round(mae, 6)

# Output results
print(f'Random Forest. Mean Squared Error: {mse_rounded}')
print(f'Random Forest. R²: {r2_rounded}')
print(f'Random Forest. Mean Absolute Error: {mae_rounded}')

### DT

In [ ]:
# Pipeline is a short construction instead of below diabetes_project
# pipeline = Pipeline([
#     ('scaler', StandardScaler()),
#     ('model', DecisionTreeRegressor(max_depth=22, min_samples_leaf=2, min_samples_split=2, random_state=42))
# ])
model = DecisionTreeRegressor(max_depth=22, min_samples_leaf=2, min_samples_split=2, random_state=42)
model.fit(X_final, y_final)

y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

mse_rounded = round(mse, 6)
r2_rounded = round(r2, 6)
mae_rounded = round(mae, 6)

# Output results
print(f'Decision Tree. Mean Squared Error: {mse_rounded}')
print(f'Decision Tree. R²: {r2_rounded}')
print(f'Decision Tree. Mean Absolute Error: {mae_rounded}')

### KNN

In [ ]:
model = KNeighborsRegressor(weights='distance', n_neighbors=7, metric='manhattan')
model.fit(X_final, y_final)

y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

mse_rounded = round(mse, 6)
r2_rounded = round(r2, 6)
mae_rounded = round(mae, 6)

# Output results
print(f'KNN. Mean Squared Error: {mse_rounded}')
print(f'KNN. R²: {r2_rounded}')
print(f'KNN. Mean Absolute Error: {mae_rounded}')

### XGB

In [ ]:
model = XGBRegressor(n_estimators=800, subsample=1.0, reg_lambda=1, reg_alpha=0.1, max_depth=14, learning_rate=0.05,
                     gamma=0, colsample_bytree=0.6)
model.fit(X_final, y_final)

y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

mse_rounded = round(mse, 6)
r2_rounded = round(r2, 6)
mae_rounded = round(mae, 6)

# Output results
print(f'XGB. Mean Squared Error: {mse_rounded}')
print(f'XGB. R²: {r2_rounded}')
print(f'XGB. Mean Absolute Error: {mae_rounded}')

### LR

In [ ]:
model = LinearRegression()
model.fit(X_final, y_final)

y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

mse_rounded = round(mse, 6)
r2_rounded = round(r2, 6)
mae_rounded = round(mae, 6)

# Output results
print(f'Random Forest. Mean Squared Error: {mse_rounded}')
print(f'Random Forest. R²: {r2_rounded}')
print(f'Random Forest. Mean Absolute Error: {mae_rounded}')

### Augmentation data (FOMA)

In [ ]:
# def foma_augmentation(X, y, alpha=0.1, n_samples=100):
#     idx = np.random.choice(len(X), size=n_samples, replace=True)
#
#     X_selected = X[idx]
#     y_selected = y.iloc[idx].values
#
#     noise = alpha * np.random.randn(*X_selected.shape)
#     X_augmented = X_selected + noise
#
#     y_augmented = y_selected + alpha * np.random.randn(n_samples)
#
#     return np.vstack([X, X_augmented]), np.hstack([y, y_augmented])
#
#
# X_train_aug, y_train_aug = foma_augmentation(X_train_scaled, y_train)
#
# model = DecisionTreeRegressor(max_depth=22, min_samples_leaf=2, min_samples_split=2, random_state=42)
# model.fit(X_train_aug, y_train_aug)
#
# y_pred = model.predict(X_test_scaled)
#
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)
#
# print(f'Decision Tree. Mean Squared Error: {round(mse, 6)}')
# print(f'Decision Tree. R²: {round(r2, 6)}')
# print(f'Decision Tree. Mean Absolute Error: {round(mae, 6)}')

### Augmentation data (FOMA)

In [ ]:
# def foma_augmentation(X, y, alpha=0.1, n_samples=100):
#     idx = np.random.choice(len(X), size=n_samples, replace=True)
#
#     X_selected = X[idx]
#     y_selected = y.iloc[idx].values
#
#     noise = alpha * np.random.randn(*X_selected.shape)
#     X_augmented = X_selected + noise
#
#     y_augmented = y_selected + alpha * np.random.randn(n_samples)
#
#     return np.vstack([X, X_augmented]), np.hstack([y, y_augmented])
#
#
# X_train_aug, y_train_aug = foma_augmentation(X_train_scaled, y_train)
#
# model = RandomForestRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=1, random_state=42)
# model.fit(X_train_aug, y_train_aug)
#
# y_pred = model.predict(X_test_scaled)
#
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)
#
# print(f'Random Forest. Mean Squared Error: {round(mse, 6)}')
# print(f'Random Forest. R²: {round(r2, 6)}')
# print(f'Random Forest. Mean Absolute Error: {round(mae, 6)}')

### DT model creation, training and evaluation (with augmentation)

In [ ]:
# model = DecisionTreeRegressor(max_depth=22, min_samples_leaf=2, min_samples_split=2, random_state=42)
# model.fit(X_train_scaled, y_train)
# # Prediction and model evaluation
# y_pred = model.predict(X_test_scaled)
#
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)
#
# mse_rounded = round(mse, 6)
# r2_rounded = round(r2, 6)
# mae_rounded = round(mae, 6)
#
# # Output results
# print(f'Decision Tree. Mean Squared Error: {mse_rounded}')
# print(f'Decision Tree. R²: {r2_rounded}')
# print(f'Decision Tree. Mean Absolute Error: {mae_rounded}')

### Without augmentation

## DT

In [ ]:
# model = DecisionTreeRegressor(max_depth=22, min_samples_leaf=2, min_samples_split=2, random_state=42)
# model.fit(X_train_scaled, y_train)
# y_pred = model.predict(X_test_scaled)
#
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)
#
# mse_rounded = round(mse, 6)
# r2_rounded = round(r2, 6)
# mae_rounded = round(mae, 6)
#
# # Output results
# print(f'Decision Tree. Mean Squared Error: {mse_rounded}')
# print(f'Decision Tree. R²: {r2_rounded}')
# print(f'Decision Tree. Mean Absolute Error: {mae_rounded}')

### RF

In [ ]:
# model = RandomForestRegressor(n_estimators=200, min_samples_split=2, min_samples_leaf=1, random_state=42)
# model.fit(X_train_scaled, y_train)
#
# # Prediction and model evaluation
# y_pred = model.predict(X_test_scaled)
#
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)
#
# mse_rounded = round(mse, 6)
# r2_rounded = round(r2, 6)
# mae_rounded = round(mae, 6)
#
# # Output results
# print(f'Random Forest. Mean Squared Error: {mse_rounded}')
# print(f'Random Forest. R²: {r2_rounded}')
# print(f'Random Forest. Mean Absolute Error: {mae_rounded}')